<a href="https://colab.research.google.com/github/Armando5347/chatbot-imitador/blob/main/prueba_gpt2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Otro intento de entrenamiento con GPT2 y el corpus de emilio

In [ ]:
!pip install transformers[torch] numpy torch huggingface_hub transformers sentencepiece datasets evaluate[dev]
from datasets import Dataset, DatasetDict, load_dataset
import json
import evaluate
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForLanguageModeling

In [ ]:
# Cargar el archivo JSON
archivo_json = "corpus_rawV5_emilio.json"
with open(archivo_json, "r", encoding="utf-8") as f:
    datos = json.load(f)

# Convertir a un Dataset de Hugging Face
dataset = Dataset.from_list(datos)
print(dataset)

Dataset({
    features: ['input', 'output'],
    num_rows: 4605
})


In [ ]:
# Modelo y tokenizador preentrenado
nombre_modelo = "gpt2"  # Cambia esto al modelo que deseas usar
tokenizer = AutoTokenizer.from_pretrained(nombre_modelo)
# Asignar el token de padding al token de fin de secuencia
tokenizer.pad_token = tokenizer.eos_token

# Dividir el dataset en entrenamiento (80%) y pruebas (20%)
dataset_split = dataset.train_test_split(test_size=0.2, seed=42)
dataset_train = dataset_split["train"]
dataset_test = dataset_split["test"]

# Revisar las divisiones
print(f"Entrenamiento: {len(dataset_train)} ejemplos")
print(f"Pruebas: {len(dataset_test)} ejemplos")

# Agregar formato personalizado
def preparar_ejemplo(ejemplo):

    texto = f"Pregunta: {ejemplo['input']}\nRespuesta: {ejemplo['output']}"
    return tokenizer(
        texto,
        truncation=True,
        padding="max_length",
        max_length=1024
    )

# Tokenizar el dataset
dataset_tokenizado = dataset_train.map(preparar_ejemplo, batched=False)
print(dataset_tokenizado)

# Configurar un Data Collator para padding dinámico, ya que no todos los mensajes tienen la misma longitud
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer,
                                                 mlm=False)

#cargar el modelo
model = AutoModelForCausalLM.from_pretrained(nombre_modelo)

#preparar argumentos de entrenamiento
training_args = TrainingArguments(
    output_dir="./chatbot_entrenamientos",
    per_device_train_batch_size=4, #cambiar a 2 si ven que consume mucha RAM
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=8,
    gradient_checkpointing=True,
    num_train_epochs=2,
    learning_rate=5e-5,
    logging_dir="./logs",
    save_steps=50,
    logging_steps=50,
    fp16=True,  # Usa FP16 si es compatible con tu GPU
    save_total_limit=1,# Mantén solo el checkpoint más recientes
    report_to="none", #para que no pida inicios de sesión ni nada
    max_steps=1000
)


Entrenamiento: 3684 ejemplos
Pruebas: 921 ejemplos


Map:   0%|          | 0/3684 [00:00<?, ? examples/s]

Dataset({
    features: ['input', 'output', 'input_ids', 'attention_mask'],
    num_rows: 3684
})


In [ ]:
# Definir el Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_tokenizado, #_tokenizado,  # Usa el dataset tokenizado
    tokenizer=tokenizer,
    data_collator=data_collator,
)

# Entrenar el modelo
trainer.train() #aquí igual como argumento meter lo de: resume_from_checkpoint=True
#guardar lo que salga o no
model.save_pretrained("./chatbot_final")
tokenizer.save_pretrained("./chatbot_final")

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
import shutil
from google.colab import files

# Ruta de la carpeta a comprimir
folder_checkpoint = "./chatbot_entrenamientos"
folder_final = "./chatbot_final"

# Nombre del archivo ZIP
output_zip_checkpoint = "chatbot_entrenamientos.zip"
output_zip_final = "chatbot_finals.zip"

# Comprimir la carpeta
shutil.make_archive(output_zip_checkpoint.replace('.zip', ''), 'zip', folder_checkpoint)
shutil.make_archive(output_zip_final.replace('.zip', ''), 'zip', folder_final)

# Descargar el archivo comprimido
files.download(output_zip_checkpoint + '.zip')
files.download(output_zip_final + '.zip')


Para probar al final

In [12]:
# Cargar el modelo entrenado, si ya lo tienes
model_final = AutoModelForCausalLM.from_pretrained("./chatbot_final")
#tokenizer_final = AutoTokenizer.from_pretrained("./chatbot_final")

# Recargar gpt2
model_original = AutoModelForCausalLM.from_pretrained("gpt2")
tokenizer_original = AutoTokenizer.from_pretrained("gpt2")

#Para el gpt2, congelar parametros, no se si tambien valga para el nuestro,s pongo que si
for param in model.parameters():
    param.requires_grad = False

# Generar texto,
def generar_respuesta(prompt, model, tokenizer):
    entrada = f"Pregunta: {prompt}\nRespuesta:"
    entrada_tokenizada = tokenizer(entrada,
                                   return_tensors="pt",
                                   padding=True,         # Asegura el padding si es necesario
                                  truncation=True,      # Trunca si el texto es muy largo
                                  max_length=1024       # Límite del modelo
                                   )
    salida_ids = model.generate(input_ids=entrada_tokenizada["input_ids"],
                                attention_mask=entrada_tokenizada["attention_mask"],
                                max_length=128,
                                num_beams=5,
                                early_stopping=True)
    return tokenizer.decode(salida_ids[0], skip_special_tokens=True)

# Probar con un ejemplo (gpt2 - baseline)
#respuesta = generar_respuesta("¿Qué hora es?", model, tokenizer)
#print(respuesta)

# Probar con un ejemplo (con el bueno)
#respuesta = generar_respuesta("¿Qué hora es?", model_final, tokenizer_final)
#print(respuesta)

# Cargar la métrica ROUGE
rouge = evaluate.load("rouge")

# Referencias y predicciones de ejemplo
# Comparar respuestas generadas con las esperadas
resultados_original = []
resultados_final = []
referencias = []
for ejemplo in dataset_test:
    prompt = ejemplo["input"]
    respuesta_esperada = ejemplo["output"]
    respuesta_generada_orignal = generar_respuesta(prompt, model_original, tokenizer_original)
    respuesta_generada_final = generar_respuesta(prompt, model_final, tokenizer_original)

    referencias.append(respuesta_esperada)

    resultados_original.append(respuesta_generada_orignal)
    resultados_final.append(respuesta_generada_final)

# Calcular la métrica ROUGE
metricas = rouge.compute(predictions=predicciones, references=referencias)

# Mostrar resultados
print(metricas)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

KeyboardInterrupt: 